In [3]:
import os
current_dir = os.getcwd()
csv_dir = os.path.join(current_dir,'每日各站進出站人數')
files_and_dirs = os.listdir(csv_dir)
files_and_dirs

['每日各站進出站人數2022.csv',
 '每日各站進出站人數2023.csv',
 '每日各站進出站人數20190423-20191231.csv',
 '每日各站進出站人數2021.csv',
 'manifest.csv',
 '每日各站進出站人數2020.csv',
 'schema.csv']

In [5]:
for filename in files_and_dirs:
    if '每日各站進出站人數' in filename:
        print(filename)

每日各站進出站人數2022.csv
每日各站進出站人數2023.csv
每日各站進出站人數20190423-20191231.csv
每日各站進出站人數2021.csv
每日各站進出站人數2020.csv


In [8]:
all_csv_files = [filename for filename in files_and_dirs if '每日各站進出站人數' in filename]

In [10]:
sorted_csv_files = sorted(all_csv_files)

In [20]:
import pandas as pd
import json

def parse_data(filename):
    df1 = pd.read_csv(filename)
    df2 = df1.rename(columns={"trnOpDate":"乘車日",
            "staCode":"車站代碼",
            "gateInComingCnt":"進站人數",
            "gateOutGoingCnt":"出站人數"})
    with open('車站基本資料集.json',encoding='utf-8') as file:
        json_data = json.load(file)
        stations_info = pd.DataFrame(json_data,columns=['stationCode','stationName'])
        stations_info = stations_info.astype({'stationCode':'int32'})
    df3 = pd.merge(df2,stations_info,left_on='車站代碼',right_on='stationCode')
    df4 = df3.rename(columns={'stationName':'車站名稱'})
    df5 = df4.reindex(columns=['乘車日','車站名稱','進站人數','出站人數'])
    df5['乘車日'] = pd.to_datetime(df5['乘車日'].astype(str))
    return df5
    

    
all_final_dfs = [] 
for filename in sorted_csv_files:
    abs_filename = os.path.join(csv_dir,filename)
    all_final_dfs.append(parse_data(abs_filename))
df_done = pd.concat(all_final_dfs)
df_done.info()
    

<class 'pandas.core.frame.DataFrame'>
Index: 406761 entries, 0 to 86608
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   乘車日     406761 non-null  datetime64[ns]
 1   車站名稱    406761 non-null  object        
 2   進站人數    406761 non-null  int64         
 3   出站人數    406761 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 15.5+ MB


In [21]:
df_done

,乘車日,車站名稱,進站人數,出站人數
0,2019-04-23,基隆,8442,7743
1,2019-04-23,三坑,1394,1348
2,2019-04-23,八堵,2770,2423
3,2019-04-23,七堵,6113,6335
4,2019-04-23,百福,2680,2726
...,...,...,...,...
86604,2023-12-31,瑞芳,7916,8252
86605,2023-12-31,海科館,164,195
86606,2023-12-31,八斗子,652,720
86607,2023-12-31,四腳亭,1526,656
